In [15]:
import pandas as pd
from pathlib import Path
import sys
import numpy as np
import dask.dataframe as dd

PATH = '/Users/danil/Documents/github/jooble'
sys.path.append(str(PATH))

требования: 

чтобы читался файл на 50 mln строк - dask

чтобы удобно было добавлть новые паттерны учитывая первое значение в фиче

использование другой нормализации кроме z 

1. открываем train.tsc при помощи (pandas -> dask -> multiprocess. compare)

2. разбиваем фичи на колонки

3. треним нормализатор (сначало просто стандартный нормализатор -> с учетом набора -> делаем удобную добавку в классах)

    - сохраняем нормализатор (если нужно)
    

4. открываем test.tsc

5. разбиваем фичи на колонки

6. оцениваем max_feature_2_index

7. оцениваем max_feature_2_abs_mean_diff

8. нормализируем

9. переименовуем feature_2 на feature_2_stand

10. сохраняем test_proc.tsc c 

    - id_job, 

    - feature_2_stand_{i} double, 

    - max_feature_2_index int - индекс самой большой фичи у этой работы, 
    - max_feature_2_abs_mean_diff - абсолютное отклонение признака с индексом max_feature_2_index от его среднего значения 
mean(feature_2_{max_feature_2_index})

Double – это 64-битная точность

1. открываем train.tsc при помощи (dask -> multiprocess. compare)

    - сохраняем нормализатор (если нужно)

Double – это 64-битная точность

# open data

In [2]:
train = dd.read_csv(PATH + '/data/raw/train.tsv', sep ='\t')
train = pd.concat([train]*300)

# prepare features

In [39]:
import re

In [44]:
re.findall(r'([a-z])', 'a,b,')

['a', 'b']

In [57]:
s = pd.Series(['a,c,', 'b,c,', 'c,c,'])
s.str.extractall(r'([a-z])')

0
  match   
0 0      a
  1      c
1 0      b
  1      c
2 0      c
  1      c

In [48]:
s = pd.Series(['ac', 'bc', 'cc'])
s.str.extract(r'([a-z])')

,0
0,a
1,b
2,c


In [36]:
train

,id_job,features
npartitions=1,,
,int64,object
,...,...


In [25]:
features = train['features'].str.split(',',256, expand=True, meta = ([int]*256))

TypeError: split() got an unexpected keyword argument 'meta'

In [22]:
features.compute()

ValueError: The columns in the computed data do not match the columns in the provided metadata
  Extra:   [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 2

In [3]:
train_feautres = train.assign(**{str(i): train.features.map(lambda x, i=i: int(x.split(',')[i]), meta=pd.Series()) 
                 for i in list(range(257))})
train_feautres = train_feautres.drop('features', 1)

/Users/danil/Library/Caches/pypoetry/virtualenvs/jooble-NhR7ocpJ-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [4]:
class preprocess:
    
    def __init__(self, normalization = 'z-score'):
        
        self.normalization = normalization
        
    def get_features_num(self, X):
        
        self.factor = train_feautres.iloc[:,1].compute().value_counts().index[0]
        if self.factor == 2:
            self.features_num = 256
        # in case of selecting special dtype, further we can select only specific column type
            # self.dtypes = int
        # here wa can add new factor check for features selection or checks
        # or redefine normalization
            # self.normalization = 'z-score'

            
    def fit_scaler(self, X): 
        
        X = X.iloc[:, 2:2 + self.features_num] #.select_dtypes(include = self.dtypes)
        
        if self.normalization == 'z-score':
            self.mean  = X.mean(axis = 0)
            self.sigma = X.std(axis = 0)
            
        elif self.normalization == 'min-max-scaler':
            self.min = X.min(axis = 0)
            self.max = X.max(axis = 0)
            
            
    def transform_with_scaler(self, X):
        
        X_to_transform = X.iloc[:, 2:2 + self.features_num] #.select_dtypes(include = self.dtypes)
        
        if self.normalization == 'z-score':
            X_to_transform = (X_to_transform - self.mean) / self.sigma
            
        elif self.normalization == 'min-max-scaler':
            X_to_transform = (X_to_transform - self.min) / (self.max - self.min)
        
        X[list(map(str, range(1, self.features_num + 1)))] = X_to_transform
        
        return X
    
    
    def add_max_feature_index(self, X):
        
        X['max_feature_2_index'] = X.iloc[:, 2: 2+ self.features_num].values.argmax(axis = 1)
        
        return X
    
    
    def add_max_feature_2_abs_mean_diff(self, X):
                        
        max_values = X.iloc[:, 2: 2+ self.features_num].values.max(axis = 1).compute()
        mean_values = X.iloc[:, 2:2+ self.features_num].values.mean(axis = 0).compute()
        
        if 'max_feature_2_index' in X.columns:
            indexes = X['max_feature_2_index']
        else:
            indexes = self.add_max_feature_index(X)['max_feature_2_index'].compute()
        
        max_feature_2_abs_mean_diff = dd.from_array(np.abs(max_values - mean_values[indexes]))

        X = X.merge(max_feature_2_abs_mean_diff.to_frame(name = 'max_feature_2_abs_mean_diff'))

        return X
    
    
    


# fit z scaler

In [5]:
prep = preprocess()

In [6]:
prep.get_features_num(train_feautres)

In [7]:
prep.fit_scaler(train_feautres)

# open test data

In [8]:
test = dd.read_csv(PATH + '/data/raw/test.tsv', sep ='\t')

# prepare features

In [9]:
test_feautres = test.assign(**{str(i): test.features.map(lambda x, i=i: int(x.split(',')[i]), meta=pd.Series()) 
                 for i in list(range(257))})
test_feautres = test_feautres.drop('features', 1)

/Users/danil/Library/Caches/pypoetry/virtualenvs/jooble-NhR7ocpJ-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


# max_feature_2_index

In [10]:
test_feautres = prep.add_max_feature_index(test_feautres)

# max_feature_2_abs_mean_diff

In [11]:
test_feautres = prep.add_max_feature_2_abs_mean_diff(test_feautres)

# feature_2_stand transform

In [13]:
test_feautres = prep.transform_with_scaler(test_feautres)

test_feautres = test_feautres.compute(scheduler='processes')

test_feautres = test_feautres.rename(columns = {str(i):f'feature_2_stand_{i}' for i in range(1, 1+ prep.features_num)})

test_feautres = test_feautres.drop(columns=['0'])

test_feautres.head()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/danil/Library/Caches/pypoetry/virtualenvs/jooble-NhR7ocpJ-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-c9c9fbb543ce>", line 1, in <module>
    test_feautres = prep.transform_with_scaler(test_feautres)
  File "<ipython-input-4-a45599f7a084>", line 37, in transform_with_scaler
    X_to_transform = (X_to_transform - self.mean) / self.sigma
  File "/Users/danil/Library/Caches/pypoetry/virtualenvs/jooble-NhR7ocpJ-py3.7/lib/python3.7/site-packages/pandas/core/ops/__init__.py", line 759, in f
    other = _align_method_FRAME(self, other, axis)
  File "/Users/danil/Library/Caches/pypoetry/virtualenvs/jooble-NhR7ocpJ-py3.7/lib/python3.7/site-packages/pandas/core/ops/__init__.py", line 676, in _align_method_FRAME
    right = to_series(right)
  File "/Users/danil/Library/Caches/pypoetry/virtualenvs/jooble-NhR7ocpJ-py3.7/lib

KeyboardInterrupt: 

In [14]:
test_feautres.head()

,id_job,feature_2_stand_1,feature_2_stand_2,feature_2_stand_3,feature_2_stand_4,feature_2_stand_5,feature_2_stand_6,feature_2_stand_7,feature_2_stand_8,feature_2_stand_9,...,feature_2_stand_249,feature_2_stand_250,feature_2_stand_251,feature_2_stand_252,feature_2_stand_253,feature_2_stand_254,feature_2_stand_255,feature_2_stand_256,max_feature_2_index,max_feature_2_abs_mean_diff
0,-9168029089769934451,0.768653,0.234160,0.719948,0.696445,-0.576554,0.802627,0.258123,0.706405,0.807730,...,0.829087,0.772043,0.120428,0.802338,-0.501696,0.184457,0.797741,0.730849,161,154.17
1,-9167993139315005259,-0.496477,0.234160,-0.162465,0.092886,0.583929,-0.398570,0.293919,0.030500,-0.419272,...,-0.189802,0.021290,0.438424,0.200994,0.419100,0.468539,-0.005891,-0.275585,161,154.17
2,-9167993136660569470,-0.501899,0.234160,-0.447115,-0.091333,0.419070,-0.131637,0.272442,-0.225248,-0.333114,...,-0.358190,-0.323519,0.378799,-0.028090,0.436243,0.305192,-0.277595,-0.890293,161,154.17
3,-9167993126042826314,-0.688054,0.232376,-0.874089,-0.563999,0.545139,-0.622126,0.458579,-0.757622,-0.814408,...,-0.894747,-0.702563,0.546315,-1.033512,0.480323,0.496947,-0.889886,-1.016850,161,154.17
4,-9167914043308884846,-1.004337,-0.204625,-1.263110,-1.143318,0.053792,-0.919089,0.532557,-1.723201,-1.574970,...,-1.034594,-1.766334,0.421388,-1.504407,0.284409,0.520621,-1.379719,-0.347903,203,21.32


# save

In [287]:
test_feautres.to_csv(PATH + '/data/processed/test_proc.tsv', sep ='\t', index = False)

In [288]:
pd.read_csv(PATH + '/data/processed/test_proc.tsv', sep ='\t')

,id_job,feature_2_stand_1,feature_2_stand_2,feature_2_stand_3,feature_2_stand_4,feature_2_stand_5,feature_2_stand_6,feature_2_stand_7,feature_2_stand_8,feature_2_stand_9,...,feature_2_stand_249,feature_2_stand_250,feature_2_stand_251,feature_2_stand_252,feature_2_stand_253,feature_2_stand_254,feature_2_stand_255,feature_2_stand_256,max_feature_2_index,max_feature_2_abs_mean_diff
0,-9168029089769934451,0.768653,0.234160,0.719948,0.696445,-0.576554,0.802627,0.258123,0.706405,0.807730,...,0.829087,0.772043,0.120428,0.802338,-0.501696,0.184457,0.797741,0.730849,161,154.17
1,-9167993139315005259,-0.496477,0.234160,-0.162465,0.092886,0.583929,-0.398570,0.293919,0.030500,-0.419272,...,-0.189802,0.021290,0.438424,0.200994,0.419100,0.468539,-0.005891,-0.275585,161,154.17
2,-9167993136660569470,-0.501899,0.234160,-0.447115,-0.091333,0.419070,-0.131637,0.272442,-0.225248,-0.333114,...,-0.358190,-0.323519,0.378799,-0.028090,0.436243,0.305192,-0.277595,-0.890293,161,154.17
3,-9167993126042826314,-0.688054,0.232376,-0.874089,-0.563999,0.545139,-0.622126,0.458579,-0.757622,-0.814408,...,-0.894747,-0.702563,0.546315,-1.033512,0.480323,0.496947,-0.889886,-1.016850,161,154.17
4,-9167914043308884846,-1.004337,-0.204625,-1.263110,-1.143318,0.053792,-0.919089,0.532557,-1.723201,-1.574970,...,-1.034594,-1.766334,0.421388,-1.504407,0.284409,0.520621,-1.379719,-0.347903,203,21.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-9163569547596238325,-0.077177,0.234160,-0.263674,0.075918,0.344721,0.095256,0.324942,-0.467948,-0.229131,...,0.141266,-0.113210,0.520762,-0.231720,0.507262,0.433029,-0.059467,-0.143000,161,154.17
96,-9163525656344858443,-0.044645,0.232376,-0.655858,-0.445227,0.661510,-0.542046,0.532557,-1.905878,-1.159038,...,-1.091675,-0.223255,0.475334,-0.578527,0.529302,0.475641,-0.824830,-1.034930,161,154.17
97,-9163472054053566485,-1.315198,0.218107,-2.031664,-1.986846,0.066722,-3.651813,0.379829,-3.205495,-4.400342,...,-4.773373,-2.519533,0.106232,-1.848032,0.152167,0.437764,-1.942261,-0.576912,161,154.17
98,-9163470125725648647,0.849983,0.235944,0.799018,0.754619,0.173396,0.926084,0.513466,0.755989,0.944393,...,0.923269,0.825843,0.512244,0.865973,0.441141,0.653192,0.874277,0.875487,1,172.12
